# Process Carraro et al.

## Set up Env

In [1]:
import os
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix

In [2]:
import liana as li

In [3]:
from prep_utils import filter_samples, filter_celltypes

In [4]:
groupby = 'major'
sample_key = 'orig.ident'
condition_key = 'type'
batch_key = 'lab'

min_cells_per_sample = 700
sample_zcounts_max = 3
sample_zcounts_min = -2

# set filtering parameters
min_cells = 10 # min number of cells per cell type
min_samples = 5 # min number of samples that pass the threshold per cell type

### Load data

In [ ]:
adata = sc.read_h5ad(os.path.join('data', "carraro.h5ad"))
adata

Filter samples, cell types, and genes

In [ ]:
adata = filter_samples(adata, sample_key, condition_key, min_cells_per_sample, sample_zcounts_max, sample_zcounts_min)

In [ ]:
adata = filter_celltypes(adata=adata, groupby=groupby, sample_key=sample_key, min_cells=min_cells, min_samples=min_samples)

In [ ]:
# Remove genes expressed in few cells, normalize
sc.pp.filter_genes(adata, min_cells=30)
adata

In [ ]:
# adata.X = sc.pp.combat(adata, key=batch_key)

### Normalize

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
li.mt.rank_aggregate.by_sample(adata, groupby=groupby, use_raw=False, sample_key=sample_key, verbose=True, n_perms=None)

In [ ]:
adata.write_h5ad(os.path.join('data', "carraro_processed.h5ad"))

## Classify

In [5]:
from classify_utils import run_classifier, run_mofatalk, run_tensor_c2c, NestedDict
from sklearn.model_selection import StratifiedKFold

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [6]:
adata = sc.read_h5ad(os.path.join('data', "carraro_processed.h5ad"), backed='r')

In [7]:
methods = li.mt.show_methods()
# in case a method is missing Magnitude Score, use Specificity Score
methods['score_key'] = methods["Magnitude Score"].fillna(methods["Specificity Score"])
# remove Geometric Mean	method
methods = methods[methods['Method Name'] != 'Geometric Mean']
# drop duplicated scores (expr_prod for NATMI & Connectome)
methods = methods.drop_duplicates(subset=['Method Name', 'score_key'])
methods = methods[['Method Name', 'score_key']]

In [8]:
adata.uns['mofa_res'] = NestedDict()
adata.uns['tensor_res'] = NestedDict()
adata.uns['auc'] = pd.DataFrame(columns=['reduction_name', 'score_key', 'fold', 'auc', 'tpr', 'fpr', 'test_split', 'train_split'])
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
for score_key in methods['score_key'][:2]:
    print(f"Creating views with: {score_key}")

    run_mofatalk(adata=adata, score_key=score_key, sample_key=sample_key, condition_key=condition_key, batch_key=batch_key)
    run_classifier(adata=adata, reduction_name='mofa_res', skf=skf, score_key=score_key)
    
    run_tensor_c2c(adata=adata, score_key=score_key, sample_key=sample_key, condition_key=condition_key)
    run_classifier(adata=adata, reduction_name='tensor_res', skf=skf, score_key=score_key)


In [ ]:
adata.uns['mofa_res']

In [ ]:
list(skf.split(adata.uns['mofa_res'].X_0['lr_means'], adata.uns['mofa_res'].y_0['lr_means']))

In [ ]:
adata.uns['auc'].sort_values('fold', ascending=False).head(10)

In [ ]:
# adata.uns['auc'].to_csv(os.path.join('data', 'results', "kuppe.csv"), index=False)